<a href="https://colab.research.google.com/github/azizrah15/Praktisi-Pert15/blob/main/NLP%20bro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk pandas Sastrawi

In [ ]:
# Import libraries
import pandas as pd
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
# Download stopwords untuk bahasa Indonesia
nltk.download('stopwords')
nltk.download('punkt')

# Menambahkan stopwords bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Load data dari file Excel/CSV
# Load your dataset
file_path = ('/content/brimo_appstore_reviews.csv')
df = pd.read_csv(file_path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         466 non-null    int64 
 1   date               466 non-null    object
 2   developerResponse  112 non-null    object
 3   review             466 non-null    object
 4   rating             466 non-null    int64 
 5   isEdited           466 non-null    bool  
 6   title              466 non-null    object
dtypes: bool(1), int64(2), object(4)
memory usage: 22.4+ KB


In [ ]:
# Menampilkan data awal (cek kolom teks)
print("Data awal:\n", df.head())

Data awal:
    Unnamed: 0             date  \
0           0  1/31/2023 15:37   
1           1  1/31/2023 17:13   
2           2  1/31/2023 13:43   
3           3  1/31/2023 11:38   
4           4  1/31/2023 11:26   

                                   developerResponse  \
0                                                NaN   
1  {'id': 34631786, 'body': 'Hai Sobat BRI, jika ...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                              review  rating  isEdited  \
0  baguss cuma please transfer ke bank lain janga...       5     False   
1  Notifikasi brimo saya matikan, krna pemilihan ...       2     False   
2  Top,, Lebih mudah transaksi  menggunakan BRIMO...       5     False   
3                                 Memudahkan hidupku       5     False   
4                              Mantap keren membantu       5     False   

 

In [ ]:
df.head()

,Unnamed: 0,date,developerResponse,review,rating,isEdited,title
0,0,1/31/2023 15:37,NaN,baguss cuma please transfer ke bank lain janga...,5,False,bagus
1,1,1/31/2023 17:13,"{'id': 34631786, 'body': 'Hai Sobat BRI, jika ...","Notifikasi brimo saya matikan, krna pemilihan ...",2,False,Bijak dalam kirim push notif
2,2,1/31/2023 13:43,NaN,"Top,, Lebih mudah transaksi menggunakan BRIMO...",5,False,TRANSAKSI MUDAH DAN AMAN BRIMO
3,3,1/31/2023 11:38,NaN,Memudahkan hidupku,5,False,bagus
4,4,1/31/2023 11:26,NaN,Mantap keren membantu,5,False,Aplikasi nya mantap sekali


In [ ]:
# Step 1: Pre-processing
# Inisialisasi stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# Daftar kata untuk normalisasi
normalization_dict = {
    "gak": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "kalo": "kalau",
    "aja": "saja",
    "kok": "",
    "sbg": "sebagai",
    "yg" : "yang",
    "tsb" : "tersebut"
    # tambahkan kata lain sesuai kebutuhan
}

In [ ]:
def normalize_text(text, normalization_dict):
    # Ganti kata-kata dalam teks sesuai dengan kamus normalisasi
    for word, replacement in normalization_dict.items():
        text = re.sub(r'\b{}\b'.format(word), replacement, text)
    return text

In [ ]:
def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Normalisasi kata-kata dengan menggunakan kamus yang telah ditentukan
    text = normalize_text(text, normalization_dict)

    # Menghapus karakter non-alfanumerik (misalnya tanda baca, simbol)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower() # Convert text to lowercase
    text = re.sub(r"http\S+", "", text) # Remove URLs
    text = re.sub(r"@\w+", "", text) # Remove mentions
    text = re.sub(r"#\w+", "", text) # Remove hashtags
    text = text.strip() # Remove leading/trailing whitespaces

In [ ]:
# Fungsi untuk melakukan pre-processing pada teks
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords bahasa Indonesia dan kata yang sangat pendek
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

    # Stemming dengan Sastrawi
    tokens = [stemmer.stem(word) for word in tokens]

    # Gabungkan kembali token menjadi satu string
    return ' '.join(tokens)

# Apply pre-processing ke setiap teks dalam kolom 'text'
# Ganti 'text' dengan nama kolom yang sesuai
df['processed_text'] = df['review'].apply(preprocess_text)


In [ ]:
# Step 2: Bag of Words Representation
vectorizer_bow = CountVectorizer()
bow_matrix = vectorizer_bow.fit_transform(df['processed_text'])

# Convert BoW matrix to DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer_bow.get_feature_names_out())

print("\nBag of Words Representation:\n", bow_df)


Bag of Words Representation:
      00  000  100  10x  11  111  1111  12  14  16  ...  wkwk  wkwkk  working  \
0     0    0    0    0   0    0     0   0   0   0  ...     0      1        0   
1     0    0    0    0   0    0     0   1   0   0  ...     0      0        0   
2     0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
3     0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
4     0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
..   ..  ...  ...  ...  ..  ...   ...  ..  ..  ..  ...   ...    ...      ...   
461   0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
462   0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
463   0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
464   0    0    0    0   0    0     0   0   0   0  ...     0      0        0   
465   0    0    0    0   0    0     0   0   0   0  ...     0      0        0   

     wow

In [ ]:
# Step 3: TF-IDF Representation
vectorizer_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizer_tfidf.fit_transform(df['processed_text'])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

print("\nTF-IDF Representation:\n", tfidf_df)


TF-IDF Representation:
       00  000  100  10x   11  111  1111        12   14   16  ...  wkwk  \
0    0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.220763  0.0  0.0  ...   0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
..   ...  ...  ...  ...  ...  ...   ...       ...  ...  ...  ...   ...   
461  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
462  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
463  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
464  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   
465  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.000000  0.0  0.0  ...   0.0   

        wkwkk  working  wow        ya  yaa  yah  yang   yg  yoogbsbdndnsn  
0    0.439

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         466 non-null    int64 
 1   date               466 non-null    object
 2   developerResponse  112 non-null    object
 3   review             466 non-null    object
 4   rating             466 non-null    int64 
 5   isEdited           466 non-null    bool  
 6   title              466 non-null    object
 7   processed_text     466 non-null    object
dtypes: bool(1), int64(2), object(5)
memory usage: 26.1+ KB
